**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [2]:
ASSAY=A001_K562_WSTARRseq
ls ${FD_RES}/results/${ASSAY}/coverage/track_normalized

A001_K562_WSTARRseq.Input.mean.SUBSET.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep1.SUBSET.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep2.SUBSET.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep3.SUBSET.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep4.SUBSET.unstranded.bed.gz
A001_K562_WSTARRseq.Output.mean.SUBSET.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep1.SUBSET.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep2.SUBSET.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep3.SUBSET.unstranded.bed.gz
A001_K562_WSTARRseq.pLog2FC.mean.SUBSET.unstranded.bed.gz


In [3]:
FD_INP=${FD_RES}/results/${ASSAY}/coverage/track_normalized
FP_INP1=$(ls ${FD_INP}/*Input*mean*SUBSET*bed.gz)
FP_INP2=$(ls ${FD_INP}/*Output*mean*SUBSET*bed.gz)
FN_INP1=$(basename ${FP_INP1})
FN_INP2=$(basename ${FP_INP2})
FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_normalized
FN_OUT=${FN_INP1/Input/pLog2FC}
FP_OUT=${FD_OUT}/${FN_OUT}

echo ${FN_INP1}
echo 
echo ${FN_INP2}
echo
echo ${FN_OUT}

A001_K562_WSTARRseq.Input.mean.SUBSET.unstranded.bed.gz

A001_K562_WSTARRseq.Output.mean.SUBSET.unstranded.bed.gz

A001_K562_WSTARRseq.pLog2FC.mean.SUBSET.unstranded.bed.gz


## RUN

In [4]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_perbase_pLog2FC_subset_WSTARR.%a.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=A001_K562_WSTARRseq
SAMPLE_GROUPS=("Input" "Output")

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output files
FD_INP=${FD_RES}/results/${ASSAY}/coverage/track_normalized
FP_INP1=$(ls ${FD_INP}/*Input*mean*SUBSET*bed.gz)
FP_INP2=$(ls ${FD_INP}/*Output*mean*SUBSET*bed.gz)
FN_INP1=$(basename ${FP_INP1})
FN_INP2=$(basename ${FP_INP2})

FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_normalized
FN_OUT=${FN_INP1/Input/pLog2FC}
FP_OUT=${FD_OUT}/${FN_OUT}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Input file: " ${FP_INP1}
echo
echo "show first few lines of file"
zcat ${FP_INP1} | head -5
echo
echo ++++++++++++++++++++++++++++++++++++++
echo "Input file: " ${FP_INP2}
echo
echo "show first few lines of file"
zcat ${FP_INP2} | head -5
echo

### execute
bedtools unionbedg -i ${FP_INP1} ${FP_INP2} \
| awk '{
    plog2fc = log($5 + 1)/log(2) - log($4 + 1)/log(2);
    print $1 "\t" $2 "\t" $3 "\t" $4 "\t" $5 "\t" plog2fc; 
  }' \
| gzip -c \
> ${FP_OUT}
  
### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Output file: " ${FP_OUT}
echo
echo "show first few lines of file"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF

Submitted batch job 29575288


## Check

In [5]:
cat ${FD_LOG}/coverage_perbase_pLog2FC_subset_WSTARR.0.txt

Hostname:           x2-07-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-16-23+10:16:33

++++++++++++++++++++++++++++++++++++++
Input file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/coverage/track_normalized/A001_K562_WSTARRseq.Input.mean.SUBSET.unstranded.bed.gz

show first few lines of file
chr11	4000000	4000001	0.153812
chr11	4000001	4000002	0.153812
chr11	4000002	4000003	0.153812
chr11	4000003	4000004	0.151309
chr11	4000004	4000005	0.151309

++++++++++++++++++++++++++++++++++++++
Input file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/coverage/track_normalized/A001_K562_WSTARRseq.Output.mean.SUBSET.unstranded.bed.gz

show first few lines of file
chr11	4000000	4000001	0.285322
chr11	4000001	4000002	0.283243
chr11	4000002	4000003	0.283243
chr11	4000003	4000004	0.28748
chr11	4000004	4000005	0.28748

++++++++++++++++++++++++++++++++++++++
Output file:  /data/reddylab/Kuei/out/proj_combeffect_encod